In [1]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import resample
from sklearn.ensemble import RandomForestRegressor
from scipy.sparse import coo_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold
from pandas import get_dummies, DataFrame
from sklearn.metrics import mean_squared_error

In [2]:


f=open('../Case Data/case1Data.txt','r')
data=f.read()
data1=data.split('\n')
arr=[]
for i in range(101):
    arr.append(data1[i].split(', '))
    


In [3]:
header=arr[0]
arr=arr[1:]
number_cols=[x[0:96] for x in arr]
class_cols=[x[96:] for x in arr]
y=np.array([x[0] for x in number_cols],dtype='float64')
x=np.array([x[1:] for x in number_cols],dtype='float64')
x_class=np.array(class_cols)

# one hot encoding





In [4]:
# # nan stuff

# c1=x_class[:,0]
# c2=x_class[:,1]
# c3=x_class[:,2]
# c4=x_class[:,3]
# c5=x_class[:,4]

# # for i in [c1,c2,c3,c4,c5]: 
# #     unique, counts = np.unique(i, return_counts=True)
# #     print (np.asarray((unique, counts)).T)

# count=0
# for i in range(101):
#     print(x_class[i])
# print(count)

In [5]:
# KNN
x_class_no_nans = x_class.copy()
def get_x_and_y(i):
    nans=np.where(i=='NaN')
    x_tmp = np.delete(x.copy(),nans,axis=0)
    y_tmp = np.delete(i.copy(),nans)
    return (x_tmp,y_tmp)

ns = range(1,25) #[1,3,5,10,20,60]
times_to_repeat = 200 
accuracy = np.zeros((x_class.shape[1],len(ns)))

for n_index, n in enumerate(ns):
    for col_index, col in enumerate(x_class.T):
        err = 0
        for loop in range(times_to_repeat):
            x_tmp,y_tmp = get_x_and_y(col)
            x_tmp, y_tmp = resample(x_tmp, y_tmp,n_samples=100)
            x_train, x_test,y_train, y_test = train_test_split(x_tmp,y_tmp,test_size=0.25)
            knn = KNeighborsClassifier(n_neighbors=n).fit(x_train,y_train)
            err += knn.score(x_test,y_test)
        err /= times_to_repeat
        accuracy[col_index, n_index] = err

In [6]:
# y_pred = knn.predict(x[nans])

for col_index, col in enumerate(x_class.T):
    best_k = np.argmax(accuracy[col_index]) + 1
    x_tmp,y_tmp = get_x_and_y(col)
    nans=np.where(col=='NaN')
    y_pred = KNeighborsClassifier(n_neighbors=best_k).fit(x_tmp,y_tmp).predict(x[nans])
    x_class_no_nans[nans,col_index] = y_pred

In [7]:
x_classes_one_hot = get_dummies(DataFrame(x_class_no_nans)).to_numpy()

x_knn = np.concatenate((x,x_classes_one_hot),axis=1)

x_one_hot = np.concatenate((x,get_dummies(DataFrame(x_class)).to_numpy()), axis=1)

In [8]:

def rmse(y_true,y_pred):
    return np.sqrt(sklearn.metrics.mean_squared_error(y_true, y_pred))
   

def train_model(model,x,y=y):
    #Linear Regression
    x_train, x_test,y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=0)
    m = model.fit(x_train, y_train)
    y_pred = m.predict(x_test)
    return rmse(y_test,y_pred)


lin_reg_rmse = train_model(LinearRegression(fit_intercept=True,normalize=True, copy_X=True, n_jobs=None), x_knn)
lin_reg_rmse_no_classes = train_model(LinearRegression(fit_intercept=True,normalize=True, copy_X=True, n_jobs=None), x)
lin_reg_rmse_no_knn = train_model(LinearRegression(fit_intercept=True,normalize=True, copy_X=True, n_jobs=None), x_one_hot)

In [9]:
times_to_repeat = 200 

def calc_accuracy(model_func, alpha, x_one=x_one_hot):
    accuracy = np.zeros((len(alpha)))
    for loop in range(times_to_repeat):
        err = 0
        for n_index, n in enumerate(alpha):
            x_train, x_test,y_train, y_test = train_test_split(x_one,y,test_size=0.25)
            model = model_func(n).fit(x_train,y_train)
            y_pred =  model.predict(x_test)
            accuracy[n_index] += (np.sum(y_test - y_pred)**2) / len(y_test)
    accuracy /= times_to_repeat
    return accuracy
lambdas = np.logspace(-5,3,100)

#Ridge Regression
ridge_accuracy = calc_accuracy(Ridge, lambdas)

#Lasso Regression 
lasso_accuracy = calc_accuracy(Lasso, lambdas)

/Users/alexandrosdorodoulis/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.5751807550883827, tolerance: 0.27079686164561195
  positive)
/Users/alexandrosdorodoulis/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.44650878464456817, tolerance: 0.23527400586902156
  positive)
/Users/alexandrosdorodoulis/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.15495654880054835, tolerance: 0.1335810254877243
  positive)
/Users/alexandrosdorodoulis/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/_coordinate_

In [19]:
#Elastic Regression
K = range(0,11,1)
accuracy = np.zeros((len(lambdas),len(K)))
for loop in range(times_to_repeat):
    err = 0
    for acc_index, acc in enumerate(K):
        for n_index, n in enumerate(lambdas):
            x_train, x_test,y_train, y_test = train_test_split(x_one_hot,y,test_size=0.25)
            model = ElasticNet(n, l1_ratio=acc).fit(x_train,y_train)
            y_pred =  model.predict(x_test)
            accuracy[n_index,acc_index] += (np.sum(y_test - y_pred)**2) / len(y_test)
accuracy /= times_to_repeat
elastic_accuracy = accuracy

In [22]:
# n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = range(1,15,3)
# # Minimum number of samples required at each leaf node
# min_samples_leaf = range(1,304,5)
# # Method of selecting samples for training each tree
# bootstrap = [True, False]# Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

In [18]:
# x_train, x_test,y_train, y_test = train_test_split(x_one_hot,y,test_size=0.25, random_state=0)

# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
# rf_random.fit(x_train, y_train)
# rf_random.best_params_

In [10]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    accuracy = 100 - (100 * np.mean(errors / test_labels))
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [11]:

# # Test if random model is actually better than the base model

# base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
# base_model.fit(x_train, y_train)
# base_accuracy = evaluate(base_model, x_test, y_test)

# best_random = rf_random.best_estimator_
# random_accuracy = evaluate(best_random, x_test, y_test)

# print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

In [12]:
# print(f'best estimator: {grid_search.best_estimator_}')

# grid_accuracy = evaluate(grid_search.best_estimator_, x_test, y_test)
# print('Improvement of {:0.2f}% over the random estimator'.format( 100 * (grid_accuracy - random_accuracy) / random_accuracy))

In [30]:
def find_random_best(hyperparams, model_to_compare=False, x=x_one_hot,y=y, K=5,estimator=RandomForestRegressor(),random_search=True):
    # cv = []
    # for tr, ts in KFold(K_outer):
    #     best_m = GridSearchCV(model, param)
    #     best_m.fit(X[tr], y[tr])
    #     s = mean_squared_error(y[ts], best_m.predict(X[ts]))
    #     cv.append(s)
    x_train, x_test,y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=0)

    rf_random = hyper_param_search_func(estimator=estimator, param_distributions=hyperparams, n_iter = 100, cv = K, verbose=2, random_state=42, n_jobs = -1) if random_search else GridSearchCV(estimator = estimator, param_grid = hyperparams, cv = 5, n_jobs = -1, verbose = 2)
    rf_random.fit(x_train, y_train)
    print(f'best paramse: {rf_random.best_params_}')
    
    base_accuracy = 0
    if model_to_compare: 
        base_accuracy = evaluate(model_to_compare, x_test, y_test)
    else:
        # Test if random model is actually better than the base model
        base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
        base_model.fit(x_train, y_train)
        base_accuracy = evaluate(base_model, x_test, y_test)

    best_random = rf_random.best_estimator_
    random_accuracy = evaluate(best_random, x_test, y_test)

    print('Improvement over model of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))
    return rf_random


In [14]:
# x_train, x_test,y_train, y_test = train_test_split(x_one_hot,y,test_size=0.25, random_state=0)

# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
# rf_random.fit(x_train, y_train)
# rf_random.best_params_
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': max_depth,
               'min_samples_split': range(1,15,3),
               'min_samples_leaf': range(1,304,5),
               'bootstrap': [True, False]}

In [15]:
# X_one_hot
random_one_hot = find_random_best(random_grid,x=x_one_hot)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   56.7s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.7min finished


best paramse: {'n_estimators': 500, 'min_samples_split': 13, 'min_samples_leaf': 6, 'max_features': 'auto', 'max_depth': 50, 'bootstrap': True}
Model Performance
Average Error: 2.8567 degrees.
Accuracy = 73.52%.
Model Performance
Average Error: 2.8470 degrees.
Accuracy = 92.08%.
Improvement of 25.24%.


In [16]:
# X_knn
random_knn = find_random_best(random_grid,x=x_knn)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   45.2s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.4min finished


best paramse: {'n_estimators': 500, 'min_samples_split': 13, 'min_samples_leaf': 6, 'max_features': 'auto', 'max_depth': 50, 'bootstrap': True}
Model Performance
Average Error: 2.8083 degrees.
Accuracy = 74.74%.
Model Performance
Average Error: 2.8497 degrees.
Accuracy = 95.27%.
Improvement of 27.47%.


In [17]:
# X
random_x = find_random_best(random_grid,x=x)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   45.9s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.6min finished


best paramse: {'n_estimators': 500, 'min_samples_split': 13, 'min_samples_leaf': 6, 'max_features': 'auto', 'max_depth': 50, 'bootstrap': True}
Model Performance
Average Error: 2.6759 degrees.
Accuracy = 66.39%.
Model Performance
Average Error: 2.9016 degrees.
Accuracy = 92.27%.
Improvement of 38.98%.


In [40]:
param_grid = {
    'min_samples_leaf': range(5,10),
    'max_depth': range(50,58),
    'min_samples_split': range(9,19),
    'n_estimators': range(410,516,5),
}

In [42]:
# #Random Forest


grid_one_hot = find_random_best(param_grid, random_one_hot.best_estimator_ , x=x_one_hot, random_search = False)
# (bootstrap=True, ccp_alpha=0.0, criterion='mse',
#                       max_depth=56, max_features='auto', max_leaf_nodes=None,
#                       max_samples=None, min_impurity_decrease=0.0,
#                       min_impurity_split=None, min_samples_leaf=7,
#                       min_samples_split=14, min_weight_fraction_leaf=0.0,
#                       n_estimators=440, n_jobs=None, oob_score=False,
#                       random_state=None, verbose=0, warm_start=False)
# best paramse: {'max_depth': 54, 'min_samples_leaf': 7, 'min_samples_split': 12, 'n_estimators': 420}

Fitting 5 folds for each of 8800 candidates, totalling 44000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   46.6s
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1961 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 2568 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 3257 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 4877 tasks      | elapsed: 11.5min
[Parallel(n_jobs=-1)]: Done 5808 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 6821 tasks      | elapsed: 16.1min
[Parallel(n_jobs=-1)]: Done 7914 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done 9089 tasks      | 

best paramse: {'max_depth': 55, 'min_samples_leaf': 5, 'min_samples_split': 11, 'n_estimators': 505}
Model Performance
Average Error: 2.8470 degrees.
Accuracy = 92.08%.
Model Performance
Average Error: 2.8513 degrees.
Accuracy = 89.07%.
Improvement over model of -3.27%.


In [41]:
grid_knn = find_random_best(param_grid, random_knn.best_estimator_ , x=x_knn, random_search = False)

Fitting 5 folds for each of 8800 candidates, totalling 44000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   48.0s
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1961 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 2568 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 3257 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 4877 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 5808 tasks      | elapsed: 14.7min
[Parallel(n_jobs=-1)]: Done 6821 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done 7914 tasks      | elapsed: 20.9min
[Parallel(n_jobs=-1)]: Done 9089 tasks      | 

best paramse: {'max_depth': 55, 'min_samples_leaf': 7, 'min_samples_split': 13, 'n_estimators': 430}
Model Performance
Average Error: 2.8497 degrees.
Accuracy = 95.27%.
Model Performance
Average Error: 2.8914 degrees.
Accuracy = 94.28%.
Improvement over model of -1.03%.


In [38]:
grid_x = find_random_best(param_grid, random_x.best_estimator_ , x=x, random_search = False)

Fitting 5 folds for each of 8000 candidates, totalling 40000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   47.0s
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 1961 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 2568 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 3257 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done 4877 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 5808 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-1)]: Done 6821 tasks      | elapsed: 18.8min
[Parallel(n_jobs=-1)]: Done 7914 tasks      | elapsed: 22.0min
[Parallel(n_jobs=-1)]: Done 9089 tasks      | 

best paramse: {'max_depth': 53, 'min_samples_leaf': 8, 'min_samples_split': 16, 'n_estimators': 430}
Model Performance
Average Error: 2.9016 degrees.
Accuracy = 92.27%.
Model Performance
Average Error: 2.8727 degrees.
Accuracy = 94.40%.
Improvement over model of 2.31%.
